In [23]:
from google.cloud import storage, bigquery
import json
import datetime

In [24]:
GCS_BUCKET = "ml_challenger_state"
GCS_STATE_FILE = "model_versions.json"

In [25]:
def load_model_versions():
    client = storage.Client()
    blob = client.bucket(GCS_BUCKET).blob(GCS_STATE_FILE)
    data = json.loads(blob.download_as_text())
    return data, blob


In [26]:
PROJECT_ID = "creature-vision"
BQ_METRICS_TABLE = "creature-vision.dog_prediction_app.prediction_metrics"
def evaluate_models(champion_version: str, challenger_version: str) -> str:
    bq = bigquery.Client(project=PROJECT_ID)
    query = f"""
        SELECT model_version, AVG(CAST(is_correct AS INT64)) AS avg_prediction_accuracy
        FROM `{BQ_METRICS_TABLE}`
        WHERE model_version IN ('{champion_version}', '{challenger_version}')
        GROUP BY model_version
    """
    results = {
        row.model_version: row.avg_prediction_accuracy
        for row in bq.query(query).result()
    }
    print("Model accuracies:", results)

    champ_score = results.get(champion_version, 0.0)
    chall_score = results.get(challenger_version, 0.0)

    return challenger_version if chall_score > champ_score else champion_version

In [ ]:
def update_model_versions(blob, data, winner: str, model_version: str):
    if data["champion"]["model_version"] != winner:
        print(f"Promoting {winner} to champion")
        data["champion"] = {
            "model_version": winner,
            "deployed_at": datetime.datetime.now().isoformat() + "Z",
        }
        
        current_version = data['challenger']['model_version']
        version_parts = current_version.strip('v').split('_')
        major, minor = map(int, version_parts)

        new_challenger_version = f"v{major}_{minor + 1}"
        print(f"New challenger: {new_challenger_version}")
        data["challenger"] = {
            "model_version": new_challenger_version,
            "deployed_at": None,
        }
        blob.upload_from_string(json.dumps(data, indent=2))
        return winner, "champion-service", new_challenger_version
    else:
        current_version = data['challenger']['model_version']
        version_parts = current_version.strip('v').split('_')
        major, minor = map(int, version_parts)

        new_challenger_version = f"v{major}_{minor + 1}"
        print(f"New challenger: {new_challenger_version}")
        data["challenger"] = {
            "model_version": new_challenger_version,
            "deployed_at": None,
        }
        blob.upload_from_string(json.dumps(data, indent=2))
        return (
            data["champion"]["model_version"],
            "challenger-service",
            new_challenger_version,
        )

In [37]:
data, blob = load_model_versions()
champion = data["champion"]["model_version"]
challenger = data["challenger"]["model_version"]

print(f"champion: {champion}\nchallenger: {challenger}")

# Step 2: Evaluate accuracy
winner = evaluate_models(champion, challenger)

# Step 3: Decide next model versions and services
model_to_update, service_to_update, new_model_version = update_model_versions(
    blob, data, winner, challenger
)
# print(model_to_update, service_to_update, new_model_version)

champion: v3_0
challenger: v3_1
Model accuracies: {'v3_1': 0.22826086956521724, 'v3_0': 0.5378091872791518}
New challenger: v3_2
